# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [2]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# load messages dataset
messages = pd.read_csv('../data/disaster_messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [4]:
# load categories dataset
categories = pd.read_csv('../data/disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
print('messages.shape: '+str(messages.shape))
print('categories.shape: '+str(categories.shape))

messages.shape: (26248, 4)
categories.shape: (26248, 2)


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [6]:
# merge datasets
df = messages.merge(categories, on=['id'], how = 'inner')
print(df.shape)

tmp = (messages['id'] == categories['id']).astype(dtype='int32')
print(tmp.loc[tmp == False])


(26386, 5)
Series([], Name: id, dtype: int32)


In [7]:
# Drop duplicates before merging
tmp = categories.drop_duplicates(subset =['id'], keep = 'first') # more entries got deleted
tmp_2 = categories.drop_duplicates(keep = 'first')

#(tmp['id']-tmp_2['id']).sum()
tmp_3 = tmp_2['id'] - tmp['id']
print(tmp_3.loc[tmp_3 != 0])

tmp.loc[tmp_2['id'] == 865]
#tmp.iloc[710]
#print('messages.shape: '+str(messages.shape))
#print('categories.shape: '+str(categories.shape))

163     NaN
710     NaN
1408    NaN
2825    NaN
3374    NaN
4341    NaN
4522    NaN
5072    NaN
5607    NaN
5697    NaN
5720    NaN
5878    NaN
6844    NaN
7037    NaN
7264    NaN
9162    NaN
10102   NaN
10822   NaN
10827   NaN
12052   NaN
12163   NaN
13206   NaN
13645   NaN
14686   NaN
14837   NaN
16086   NaN
16156   NaN
16281   NaN
16769   NaN
19625   NaN
20876   NaN
20966   NaN
22028   NaN
24040   NaN
24648   NaN
25156   NaN
Name: id, dtype: float64


,id,categories
713,865,related-0;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [8]:
# select the first row of the categories dataframe
first_row = categories['categories'].iloc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = [elem[:-2] for elem in first_row.split(';')]
print(category_colnames)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


In [9]:
new_categories = categories['categories'].str.split(';', expand = True)
new_categories = new_categories.replace(to_replace=r'.*-', value='', regex= True).astype(dtype='int32')
new_categories.columns = category_colnames
print(new_categories.head())

   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   
1        1        0      0            1             0                 0   
2        1        0      0            0             0                 0   
3        1        1      0            1             0                 1   
4        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  aid_centers  \
0                  0         0         0            0  ...            0   
1                  0         0         0            0  ...            0   
2                  0         0         0            0  ...            0   
3                  0         0         0            0  ...            0   
4                  0         0         0            0  ...            0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                    

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [10]:
# Already done in Step 3
#for column in categories:
#    # set each value to be the last character of the string
#    categories[column] = 
#    
#    # convert column from string to numeric
#    categories[column] = 
#categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [11]:
# drop the original categories column from `df`
df.drop('categories', axis=1, inplace = True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [19]:
# concatenate the original dataframe with the new `categories` dataframe
print("df.shape: {}, new_categories.shape: {}".format(df.shape, new_categories.shape))
new_df = pd.concat([df, new_categories], axis = 1)
print('new_df.shape: {}'.format(new_df.shape))
new_df.head()
df = new_df

df.shape: (26180, 4), new_categories.shape: (26248, 36)
new_df.shape: (26386, 40)


In [20]:
new_df.tail()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
26381,30261.0,The training demonstrated how to enhance micro...,NaN,news,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26382,30262.0,A suitable candidate has been selected and OCH...,NaN,news,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26383,30263.0,"Proshika, operating in Cox's Bazar municipalit...",NaN,news,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26384,30264.0,"Some 2,000 women protesting against the conduc...",NaN,news,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26385,30265.0,A radical shift in thinking came about as a re...,NaN,news,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
new_df.shape

(26386, 40)

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [22]:
# check number of duplicates
print("Number of duplicated rows: ",len(df[df.duplicated(keep = 'first')]))# Select duplicate rows except first occurrence based on all columns

Number of duplicated rows:  112


In [23]:
# drop duplicates
print(df.shape)
df = df.drop_duplicates(keep = 'first')
print(df.shape)

(26386, 40)
(26274, 40)


In [24]:
# check number of duplicates
print("Number of duplicated rows: ",len(df[df.duplicated(keep = 'first')]))

Number of duplicated rows:  0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [25]:
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df.to_sql('messages', engine, index=False, if_exists = 'replace')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.